In [7]:
from ib_insync import *
util.startLoop()
import stock

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=12)
ib.isConnected()

True

In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))


from ib_insync import *
util.startLoop()
import stock

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=12)
sx = stock.StockX(ib, 'TSLA')
sx.set_up_frame('1 day', 'ohlcv', start_date="52 weeksAgo", end_date="now")
sx.frames['1 day'].data


Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Stored data: 270 rows of data
Processing Missing data: 1 intervals
get_batch_historical_data :: lowest_interval='1 day', rth=False
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Data loaded: 274 rows of data


,open,high,low,close,volume
date,,,,,
2024-01-16,214.80,223.49,212.18,220.55,949389
2024-01-17,215.90,219.00,212.01,213.46,862280
2024-01-18,215.53,219.26,208.74,211.00,899772
2024-01-19,211.85,213.19,207.56,212.40,862805
2024-01-22,213.45,217.80,206.27,208.01,991809
...,...,...,...,...,...
2025-01-07,411.35,414.33,391.20,397.80,388126
2025-01-10,395.65,399.28,377.29,393.10,471504
2025-01-13,385.90,413.50,380.07,413.25,506292


In [2]:
import strategies.ta as ta
import strategies.preset_strats as ps
f = sx.frames['1 day']
f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=10), [{'color': 'green', 'size': 10}, {'color': 'red', 'size': 10}], chart_type = 'points')
f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=3), [{'color': 'green', 'size': 5}, {'color': 'red', 'size': 5}], chart_type = 'points')
f.add_ta(ta.Ffill(colToFfill='HP_hi_3'), {'dash': 'dot', 'color': 'green', 'width': 1}, chart_type = 'line', row=1)
f.add_ta(ta.Ffill(colToFfill='LP_lo_3'), {'dash': 'dot', 'color': 'red', 'width': 1}, chart_type = 'line', row=1)
ps.require_ta_for_all(f, pointsSpan=10)
ps.ma_ta(f, [21, 50, 200])
f.plot()
f.data.columns

Index(['open', 'high', 'low', 'close', 'volume', 'HP_hi_10', 'LP_lo_10',
       'HP_hi_3', 'LP_lo_3', 'FFILL_HP_hi_3', 'FFILL_LP_lo_3', 'ATR_50',
       'Res_1', 'Res_1_Upper', 'Res_1_Lower', 'Res_2', 'Res_2_Upper',
       'Res_2_Lower', 'Sup_1', 'Sup_1_Upper', 'Sup_1_Lower', 'Sup_2',
       'Sup_2_Upper', 'Sup_2_Lower', 'MA_cl_21', 'MA_cl_50', 'MA_cl_200'],
      dtype='object')

# Place orders using raw prices values

In [3]:
import sb_order
ox = sb_order.OrderX('Strat1', ib, 'TSLA', 'LONG')
ox.set_entry(qty=8, outsideRth=True, limitPrice=400.00)
ox.add_bracket_order(qtyPct=25, stop_price=370.00, target_price=420.00)
ox.add_bracket_order(qtyPct=25, stop_price=350.00, target_price=470.00)
ox.add_stop_order(qtyPct=50, stop_price=395.00)
# ox.place_orders(delay_between_orders=1)

621

In [4]:
ox.orders

[MarketOrder(orderId=615, action='BUY', totalQuantity=2, lmtPrice=400.0, orderRef='Strat1_Entry_1', transmit=False, outsideRth=True),
 StopOrder(orderId=616, action='SELL', totalQuantity=2, auxPrice=370.0, orderRef='Strat1_Stop_1_pct25', transmit=False, parentId=615),
 LimitOrder(orderId=617, action='SELL', totalQuantity=2, lmtPrice=420.0, orderRef='Strat1_Tget_1_pct25', parentId=615),
 MarketOrder(orderId=618, action='BUY', totalQuantity=2, lmtPrice=400.0, orderRef='Strat1_Entry_2', transmit=False, outsideRth=True),
 StopOrder(orderId=619, action='SELL', totalQuantity=2, auxPrice=350.0, orderRef='Strat1_Stop_2_pct25', transmit=False, parentId=618),
 LimitOrder(orderId=620, action='SELL', totalQuantity=2, lmtPrice=470.0, orderRef='Strat1_Tget_2_pct25', parentId=618),
 MarketOrder(orderId=621, action='BUY', totalQuantity=4, lmtPrice=400.0, orderRef='Strat1_Entry_3', transmit=False, outsideRth=True),
 StopOrder(orderId=622, action='SELL', totalQuantity=4, auxPrice=395.0, orderRef='Strat1

In [4]:
ox.cancel_orders()

# Place Orders using DataFrame and column names to get prices

In [3]:
import sb_order
import pandas as pd

    
print(f.data['Res_2'].iat[-1])
print(f.data['Res_1'].iat[-1])
print(f.data['Sup_1'].iat[-1])
print(f.data['Sup_2'].iat[-1])
print(f.data['MA_cl_50'].iat[-1])
print(f.data['FFILL_HP_hi_3'].iat[-1])


oxd = sb_order.OrderXData('Strat1', ib, 'TSLA', 'LONG')
oxd.set_entry(f.data, qty=8, outsideRth=True, limitPrice='close')
oxd.add_bracket_order(f.data, qtyPct=25, stop_price='Sup_1', target_price='FFILL_HP_hi_3')
oxd.add_bracket_order(f.data, qtyPct=25, stop_price='Sup_2', target_price='Res_1')
oxd.add_stop_order(f.data, qtyPct=50, stop_price='Sup_1')
oxd.place_orders(f.data, delay_between_orders=0)
print("Connection status:", ib.isConnected())

nan
488.54
362.8
271.0
346.6318
465.33
Stop quotas:  QtyPct: 25, Qty: 2, Quotas: 6
Stop quotas:  QtyPct: 25, Qty: 2, Quotas: 4
Stop quotas:  QtyPct: 50, Qty: 4, Quotas: 0
Connection status: True


In [4]:
oxd.modify_stop(f.data, stopNum=1, stop_price='MA_cl_21')
oxd.modify_stop(f.data, stopNum=2, stop_price='MA_cl_50')

Old price: 362.8, New price: 415.68, new > old = True
Old price: 271.0, New price: 346.63, new > old = True


In [4]:
display(oxd.get_orders_as_df())
display(oxd.get_orders_status_as_df()) # gets auto updated live when connected to IB

,parentId,orderId,clientId,permId,action,totalQuantity,orderType,ocaGroup,ocaType,orderRef,lmtPrice,auxPrice,outsideRth,tif,goodTillDate,transmit
0,0,595,12,551174510,BUY,2.0,LMT,,0,Strat1_Entry_1,403.84,0.0,True,,,False
1,595,596,12,551174511,SELL,2.0,STP,,0,Strat1_Stop_1_pct25,0.00,362.8,False,,,False
2,595,597,12,551174512,SELL,2.0,LMT,,0,Strat1_Tget_1_pct25,465.33,0.0,False,,,True
3,0,598,12,551174513,BUY,2.0,LMT,,0,Strat1_Entry_2,403.84,0.0,True,,,False
4,598,599,12,551174514,SELL,2.0,STP,,0,Strat1_Stop_2_pct25,0.00,271.0,False,,,False
5,598,600,12,551174515,SELL,2.0,LMT,,0,Strat1_Tget_2_pct25,488.54,0.0,False,,,True
6,0,601,12,551174516,BUY,4.0,LMT,,0,Strat1_Entry_3,403.84,0.0,True,,,False
7,601,602,12,551174517,SELL,4.0,STP,,0,Strat1_Stop_3_pct50,0.00,362.8,False,,,True


,orderId,status,filled,remaining,avgFillPrice,permId,parentId,lastFillPrice,clientId,whyHeld,mktCapPrice
0,595,Submitted,0.0,2.0,0.0,551174510,0,0.0,12,,0.0
1,596,PreSubmitted,0.0,2.0,0.0,551174511,595,0.0,12,"child,trigger",0.0
2,597,PreSubmitted,0.0,2.0,0.0,551174512,595,0.0,12,child,0.0
3,598,Submitted,0.0,2.0,0.0,551174513,0,0.0,12,,0.0
4,599,PreSubmitted,0.0,2.0,0.0,551174514,598,0.0,12,"child,trigger",0.0
5,600,PreSubmitted,0.0,2.0,0.0,551174515,598,0.0,12,child,0.0
6,601,Submitted,0.0,4.0,0.0,551174516,0,0.0,12,,0.0
7,602,PreSubmitted,0.0,4.0,0.0,551174517,601,0.0,12,"child,trigger",0.0


In [5]:
oxd.cancel_orders()

# Using OrderXData to update current stock positions

In [8]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))


from ib_insync import *
# util.startLoop()
import stock

ib = IB()
# ib.connect('127.0.0.1', 7496, clientId=12)
sx = stock.StockX(ib, 'TSLA')
sx.set_up_frame('1 day', 'ohlcv', start_date="52 weeksAgo", end_date="now")
sx.frames['1 day'].data

Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Stored data: 270 rows of data
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Data loaded: 270 rows of data


,open,high,low,close,volume
date,,,,,
2024-01-09,239.45,240.50,232.04,234.20,815477
2024-01-10,235.51,236.54,231.33,233.72,778458
2024-01-11,235.51,236.00,225.00,225.05,872427
2024-01-12,222.70,225.34,217.15,218.95,1023558
2024-01-16,214.80,223.49,212.18,220.55,949389
...,...,...,...,...,...
2024-12-26,465.16,465.33,451.02,454.13,508124
2024-12-27,449.37,450.00,426.50,431.66,559640
2024-12-30,419.40,427.00,415.75,417.41,411128


In [19]:
import strategies.ta as ta
import strategies.preset_strats as ps
f = sx.frames['1 hour']
f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=10), [{'color': 'green', 'size': 10}, {'color': 'red', 'size': 10}], chart_type = 'points')
f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=3), [{'color': 'green', 'size': 5}, {'color': 'red', 'size': 5}], chart_type = 'points')
f.add_ta(ta.Ffill(colToFfill='HP_hi_3'), {'dash': 'dot', 'color': 'green', 'width': 1}, chart_type = 'line', row=1)
f.add_ta(ta.Ffill(colToFfill='LP_lo_3'), {'dash': 'dot', 'color': 'red', 'width': 1}, chart_type = 'line', row=1)
ps.require_ta_for_all(f, pointsSpan=10)
ps.ma_ta(f, [21, 50, 200])
f.plot()
f.data.columns

KeyError: '1 hour'

In [ ]:
# strategies for stops.  stopX
# what to use to swicth stop metrics
# 1. rRatio - risk reward ratio.  if > 2 then swictch to using MA8 as stop
# 2. Accelertaion is > x then switch to using MA8 as stop
# 3. If price is > x% from support to resistance then switch to using MA8 as stop

sm = StopX(ls='LONG', data=f.data, entry='close')   # uses close as entry price to help compute current risk reward ratio as time moves on and prices change. can vlaues also
sm.init_stop(stopCol='Sup_1', offsetPct=1, offsetVal=0.01) # column to use as stop. shift is the number of bars to lag the stop
sm.new_stop(trig='rRatio', trigVal=2, newStop='MA_cl_21', delay=1, offsetPct=1, offsetVal=0.01)  # if rRatio is > 2 then switch to using MA_cl_21 as stop. if trig is rRatio then cumpute rRatio using entry price, init stop price and current price becaseu each rRatio is unique to the current bar and the current trade
sm.new_stop(trig='accel',  trigVal=0.5, newStop='MA_cl_8',  delay=1, offsetPct=1, offsetVal=0.01) # if acceleration is > 0.5 then switch to using MA_cl_8 as stop
sm.new_stop(trig='priceDist', trigVal=0.5, newStop='MA_cl_8',  delay=1, offsetPct=1, offsetVal=0.01) # if price is > 50% from support to resistance then switch to using MA_cl_8 as stop
sm.update_stops(f.data)
sm.get_stop(asValue=False) # returns the column name (if asValue=False) of the stop for the current bar.  gets fed back into the order manager

In [ ]:
import sb_order
import pandas as pd

    
print(f.data['Res_2'].iat[-1])
print(f.data['Res_1'].iat[-1])
print(f.data['Sup_1'].iat[-1])
print(f.data['Sup_2'].iat[-1])
print(f.data['MA_cl_50'].iat[-1])
print(f.data['FFILL_HP_hi_3'].iat[-1])


oxd = sb_order.OrderXData('Strat1', ib, 'TSLA', 'LONG')
oxd.modify_stop(f.data, stopNum=1, stop_price='MA_cl_21')
print("Connection status:", ib.isConnected())

In [5]:
portfolio = ib.portfolio()
portfolio

[PortfolioItem(contract=Stock(conId=76792991, symbol='TSLA', right='0', primaryExchange='ISLAND', currency='USD', localSymbol='TSLA', tradingClass='NMS'), position=8.0, marketPrice=413.0, marketValue=3304.0, averageCost=410.3008, unrealizedPNL=21.59, realizedPNL=0.0, account='DU1780937')]

In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

from ib_insync import *
util.startLoop()
import stock

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=12)
sx = stock.StockX(ib, 'TSLA')
sx.set_up_frame('1 day', 'ohlcv', start_date="52 weeksAgo", end_date="now")
# sx.set_up_frame('1 day', 'ohlcv', start_date="52 weeksAgo", end_date="2025-01-10")
sx.frames['1 day'].data


Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Stored data: 274 rows of data
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Data loaded: 274 rows of data


,open,high,low,close,volume
date,,,,,
2024-01-16,214.80,223.49,212.18,220.55,949389
2024-01-17,215.90,219.00,212.01,213.46,862280
2024-01-18,215.53,219.26,208.74,211.00,899772
2024-01-19,211.85,213.19,207.56,212.40,862805
2024-01-22,213.45,217.80,206.27,208.01,991809
...,...,...,...,...,...
2025-01-07,411.35,414.33,391.20,397.80,388126
2025-01-10,395.65,399.28,377.29,393.10,471504
2025-01-13,385.90,413.50,380.07,413.25,506292


Peer closed connection.


In [2]:
import pandas as pd
import strategies.ta as ta
import strategies.preset_strats as ps
import sb_prices as pricex

        
# stpx.get_stop(asValue=True)
f = sx.frames['1 day']

initType = 'accel'
initType = 'rrr'
px  = pricex.PriceX(name='Strat1', ls='LONG', includeTarget=True,
                entry  = pricex.EntryX(name='Entry1'),
                stop   = pricex.StopX(name='Stop1', longPriceCol='LoIst_lo_3', shortPriceCol='HiIst_hi_3', offsetVal=0.50, barsAgo=1),
                target = pricex.TargetX(name='Target1', longPriceCol='Res_1', shortPriceCol='Sup_1', offsetVal=0.50, barsAgo=1, rrIfNoTarget=2),
                trails = [
                    pricex.TrailX(name='Stop1', initType=initType, initTrigVal=1, barsAgo=2, longPriceCol='FFILL_LP_lo_3', shortPriceCol='FFILL_HP_hi_3', offsetVal=0.01),
                    pricex.TrailX(name='Stop2', initType=initType, initTrigVal=2, barsAgo=2, longPriceCol='MA_cl_50',      shortPriceCol='MA_cl_50',      offsetVal=0.01),
                    pricex.TrailX(name='Stop3', initType=initType, initTrigVal=3, barsAgo=2, longPriceCol='MA_cl_21',      shortPriceCol='MA_cl_21',      offsetVal=0.01),
                    # TrailX(name='Stop4', initType=initType, initTrigVal=4, barsAgo=2, priceCol='MA_cl_8', offsetVal=0.01),
                    # TrailX(name='Stop5', initType=initType, initTrigVal=6, barsAgo=2, priceCol='LoIst_lo_1', offsetVal=0.01),
                    # TrailX(name='Stop1', initType=initType, initTrigVal=2BarsAgo=1, 0, priceCol='MA_cl_50', offsetVal=0.01),
                    # TrailX(name='Stop2', initType=initType, initTrigVal=50, priceCol='FFILL_LP_lo_3', offsetVal=0.01),
                    # TrailX(name='Stop3', initType=initType, initTrigVal=70, priceCol='MA_cl_21', offsetVal=0.01),
                    # TrailX(name='Stop4', initType=initType, initTrigVal=90, priceCol='MA_cl_8', offsetVal=0.01),
                    # TrailX(name='Stop5', initType=initType, initTrigVal=100, priceCol='LoIst_lo_1', offsetVal=0.01),
                    ])
px.set_columns(f.data)
f.update_data(f.data)

ps.req_for_price_x(f, px.entryName, px.stopName, px.targetName, px.riskName, stopNameCol='StopName')
ps.require_ta_for_all(f, pointsSpan=10)
ps.ma_ta(f, [8, 21, 50, 200])


for index, row in f.data.iterrows():
    dfx = f.data.loc[:index].copy()
    if len (dfx) < 2:
        continue
    px.run_row(dfx)
    f.data.loc[index, px.entryName] = px.get_entry_price()
    f.data.loc[index, px.stopName] = px.get_stop_price()
    f.data.loc[index, px.targetName] = px.get_target_price()
    f.data.loc[index, px.riskName] = px.get_risk_reward_ratio()
    f.data.loc[index, 'StopName'] = px.get_active_stop_name()

    



    
# f.chart.refesh(f.data)
print(f.data.columns)
print(px.status)
# display(f.data[['close', 'Entry1', 'Target1', 'StopPrice', 'StopName', 'Risk1', 'ACC_close', 'FFILL_LP_lo_3']].tail(50))
f.plot(height=1200)


Index(['open', 'high', 'low', 'close', 'volume', 'Entry1', 'Stop1', 'Target1',
       'Stop3', 'Stop2', 'Risk1', 'Trace1', 'Accel1', 'Strat1_Ent',
       'Strat1_Stp', 'Strat1_Tgt', 'Strat1_Rsk', 'HP_hi_10', 'LP_lo_10',
       'HP_hi_3', 'LP_lo_3', 'ATR_14', 'ACC_close', 'FFILL_HP_hi_3',
       'FFILL_LP_lo_3', 'HiIst_hi_3', 'LoIst_lo_3', 'HiIst_hi_1', 'LoIst_lo_1',
       'ATR_50', 'Res_1', 'Res_1_Upper', 'Res_1_Lower', 'Res_2', 'Res_2_Upper',
       'Res_2_Lower', 'Sup_1', 'Sup_1_Upper', 'Sup_1_Lower', 'Sup_2',
       'Sup_2_Upper', 'Sup_2_Lower', 'MA_cl_8', 'MA_cl_21', 'MA_cl_50',
       'MA_cl_200', 'StopName'],
      dtype='object')
IN_TRADE


In [4]:
f.data['StopName'].tail(50)

date
2024-10-22         
2024-10-23    Stop1
2024-10-24    Stop1
2024-10-25    Stop1
2024-10-28    Stop1
2024-10-29    Stop1
2024-10-30    Stop1
2024-10-31    Stop1
2024-11-01    Stop1
2024-11-04    Stop1
2024-11-05    Stop1
2024-11-06    Stop1
2024-11-07    Stop1
2024-11-08    Stop1
2024-11-11    Stop1
2024-11-12    Stop1
2024-11-13    Stop1
2024-11-14    Stop1
2024-11-15    Stop1
2024-11-18    Stop1
2024-11-19    Stop1
2024-11-20    Stop1
2024-11-21    Stop1
2024-11-22    Stop1
2024-11-25    Stop1
2024-11-26    Stop1
2024-11-27    Stop1
2024-11-29    Stop1
2024-12-02    Stop1
2024-12-03    Stop1
2024-12-04    Stop1
2024-12-05    Stop1
2024-12-06    Stop1
2024-12-09    Stop1
2024-12-10    Stop1
2024-12-11    Stop1
2024-12-12         
2024-12-13    Stop1
2024-12-16    Stop1
2024-12-17    Stop1
2024-12-18    Stop1
2024-12-19    Stop1
2024-12-20    Stop1
2024-12-23    Stop1
2024-12-24    Stop1
2024-12-26    Stop1
2024-12-27    Stop1
2024-12-30    Stop1
2024-12-31    Stop1
2025-01-07    S

In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

from ib_insync import *
util.startLoop()
import stock

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=12)
sx = stock.StockX(ib, 'TSLA')
sx.set_up_frame('1 day', 'ohlcv', start_date="52 weeksAgo", end_date="now", force_download=False)
# sx.set_up_frame('1 day', 'ohlcv', start_date="52 weeksAgo", end_date="2025-01-10")
sx.frames['1 day'].data

Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Stored data: 285 rows of data
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Data loaded: 285 rows of data


,open,high,low,close,volume
date,,,,,
2024-01-16,214.80,223.49,212.18,220.55,949389
2024-01-17,215.90,219.00,212.01,213.46,862280
2024-01-18,215.53,219.26,208.74,211.00,899772
2024-01-19,211.85,213.19,207.56,212.40,862805
2024-01-22,213.45,217.80,206.27,208.01,991809
...,...,...,...,...,...
2025-01-13,385.90,413.50,380.07,413.25,506292
2025-01-14,412.70,422.64,393.00,396.04,639088
2025-01-15,395.89,422.43,394.98,421.89,401383


In [2]:
from dataclasses import dataclass
import pandas as pd
import copy
import sb_order
import sb_prices 
from sb_prices import PriceX, EntryX, StopX, TargetX, RiskX, TrailX, AccelX, TraceX, QtyX
import strategies.ta as ta
import strategies.preset_strats as ps
import my_ib_utils 
my_ib_utils.is_within_trading_hours(ib, 'TSLA', 'SMART')


@dataclass
class TradeSummery:
    status:str
    outsideRth:bool
    isOutSideRth:bool
    qtyPct:int
    qty:int
    riskAmount:float
    totalValue:float
    potentialLoss:float
    entryPrice:float
    entryLimitPrice:float
    stopPrice:float
    targetPrice:float
    entryOrderType:str
    actualEntryOrderType:str


@dataclass
class TraderX:
    name:str = 'Trader1'
    ib:IB = None
    symbol:str =''
    ls:str = 'LONG'

    def __post_init__(self):
        self.orderX = sb_order.OrderX('Strat1', self.ib, self.symbol, self.ls) # order manager
        self.pricexGroups = []  # list of priceX objects to manage multiple priceX objects
        self.tradeSummeries = [] # list of trade summeries for each priceX object

    def validate_frame(self, f):
        """Place holder for validation of the frame. 
        1. if MKT order is the frame set to force download meaning has it got the latest data
        """
        pass

    def add_entry(self, entryx:EntryX=None):
            self.entryx = entryx

    def add_stop_and_target(self, qtyPct=25, targetx:TargetX=None, initStop:StopX=None, trailingStopPrices:list[TrailX]=None):
        name  = f"{self.name}_Strat{len(self.pricexGroups)+1}"
        pricex = sb_prices.PriceX(name=name, ls=self.ls, includeTarget=True)
        pricex.entry = copy.deepcopy(self.entryx)
        pricex.stop = initStop
        if targetx: 
            pricex.target = targetx

        if trailingStopPrices: 
            pricex.trails = trailingStopPrices

        pricex.set_names()
        self.pricexGroups += [(qtyPct, pricex)]

    def add_stop(self, qtyPct=25, initStop:StopX=None, trailingStopPrices:list[TrailX]=None):
        name  = f"{self.name}_Strat{len(self.pricexGroups)+1}"
        pricex = sb_prices.PriceX(name=name, ls=self.ls, includeTarget=False)
        pricex.entry = copy.deepcopy(self.entryx)
        pricex.stop = initStop
        if trailingStopPrices: 
            pricex.trails = trailingStopPrices
        
        pricex.set_names()
        self.pricexGroups += [(qtyPct, pricex)]

    def set_columns(self, data):
        for qtyPct, pricex in self.pricexGroups:
            pricex.set_ls()
            pricex.set_columns(data)

    def set_orders(self, data=None, riskAmount=None, outsideRth=False, delay_between_orders=0):
        totalQty = 0
        potential_loss = 0
        total_value = 0
        is_rth = my_ib_utils.is_within_trading_hours(self.ib, self.symbol, 'SMART')

        for qtyPct, pricex in self.pricexGroups:
            pricex.run_row(data, is_rth)

            riskAmountPerBracket = riskAmount / 100 * qtyPct
            pricex.compute_qty(riskAmountPerBracket)
            totalQty += pricex.qty.get_qty()
            potential_loss += pricex.qty.potential_loss
            total_value += pricex.qty.total_value

            # helps to collect all the various values for the trade summary
            self.tradeSummeries += [TradeSummery(
                outsideRth=outsideRth,
                isOutSideRth= not is_rth,
                entryOrderType=pricex.entry.orderType,
                actualEntryOrderType='',
                qtyPct=qtyPct,
                qty=pricex.qty.get_qty(),
                entryPrice=pricex.entry.get_price(),
                entryLimitPrice=pricex.entry.get_limit_price(),
                stopPrice=pricex.stop.get_price(),
                targetPrice=pricex.target.get_price() if pricex.includeTarget else None,
                totalValue=pricex.qty.total_value,
                potentialLoss=pricex.qty.potential_loss,
                riskAmount=riskAmountPerBracket,
                status=pricex.status
            )]


        # display(pd.DataFrame(self.tradeSummeries))

        # set entry type to market if entry price is the close price of the current bar
        entryObj = tx.pricexGroups[0][1].entry
        entryPrice = entryObj.get_price() if entryObj.orderType in ['STP', 'STP LMT'] else None
        limitEntryPrice = entryObj.get_limit_price() if entryObj.orderType in ['LMT', 'STP LMT'] else None

        print(f"Limit Entry Price: {limitEntryPrice}, Entry Price: {entryPrice}")

        self.orderX.set_entry(
            isRth            = is_rth,
            entryOrderType   = self.entryx.orderType,  # gets set by EntryX object
            qty              = totalQty,               # gets set above by computing the total qty for all the brackets
            outsideRth       = outsideRth,             # gets set directly by the args passed in
            limitPrice       = limitEntryPrice,        # get set dierctly by the args passed in OR if outsideRth is True then it gets set by the EntryX object
            entryPrice       = entryPrice              # gets set by the EntryX object only if the entry type is STP otherwise it is None whcih means it is a market order
            )
            
        for qtyPct, pricex in self.pricexGroups:
            if pricex.includeTarget:
                entry_order, stop_order, target_order = self.orderX.add_bracket_order(qtyPct=qtyPct, stop_price=pricex.stop.get_price(), target_price=pricex.target.get_price())
                pricex.entry.ibId = entry_order.orderId
                pricex.stop.ibId = stop_order.orderId
                pricex.target.ibId = target_order.orderId
            else:
                entry_order, stop_order = self.orderX.add_stop_order(qtyPct=qtyPct, stop_price=pricex.stop.get_price())
                pricex.entry.ibId = entry_order.orderId
                pricex.stop.ibId = stop_order.orderId

    def update_filled_status(self):
        def get_remaining_by_orderid(df, order_id):
            s = df.loc[df['orderId'] == order_id, 'remaining']
            if len(s) == 0:
                return None
            return s.iloc[0]

        order_status_df = self.orderX.get_orders_status_as_df()

        for pct, pricex in self.pricexGroups:
            entry_remainng = get_remaining_by_orderid(order_status_df, pricex.entry.ibId)
            stop_remainng = get_remaining_by_orderid(order_status_df, pricex.stop.ibId)
            tget_remainng = get_remaining_by_orderid(order_status_df, pricex.target.ibId)

            #  do it this way so that when running pricex.run_row() it will trigger the has_triggered method
            pricex.entry.has_triggered(forceTrigger=entry_remainng == 0)
            pricex.stop.has_triggered(forceTrigger=stop_remainng == 0)
            pricex.target.has_triggered(forceTrigger=tget_remainng == 0)

    def update_stops(self, data):
        for qtyPct, pricex in self.pricexGroups:
            pricex.run_row(data)



    def add_ta(self, f):
        pointSpan = 10
        atrSpan = 14
        ps.ma_ta(f, [8, 21, 50, 200])
        f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=pointSpan), [{'color': 'green', 'size': 10}, {'color': 'red', 'size': 10}], chart_type = 'points')
        f.add_ta(ta.HPLP(hi_col='high', lo_col='low', span=3), [{'color': 'green', 'size': 10}, {'color': 'red', 'size': 10}], chart_type = 'points')
        f.add_ta(ta.ATR(span=atrSpan),{'dash': 'dot', 'color': 'red', 'width': 1}, chart_type = 'ine', row=1)
        f.add_ta(ta.ACC('close', fast_ma=8, slow_ma=21, max_accel=50), {'dash': 'dot', 'color': 'red', 'width': 1}, chart_type = 'line', row=3)
        f.add_ta(ta.Ffill(colToFfill='HP_hi_3'), {'dash': 'dot', 'color': 'green', 'width': 1}, chart_type = 'line', row=1)
        f.add_ta(ta.Ffill(colToFfill='LP_lo_3'), {'dash': 'dot', 'color': 'red', 'width': 1}, chart_type = 'line', row=1)
        f.add_ta(ta.LowestHighest(hi_col='high', lo_col='low', span=3), [{'color': 'green', 'size': 1}, {'color': 'red', 'size': 5}], chart_type = 'points')
        f.add_ta(ta.LowestHighest(hi_col='high', lo_col='low', span=1), [{'color': 'green', 'size': 1}, {'color': 'red', 'size': 5}], chart_type = 'points')
        f.add_ta(ta.SupRes(hi_point_col=f'HP_hi_{pointSpan}', lo_point_col=f'LP_lo_{pointSpan}', atr_col=f'ATR_{atrSpan}', tolerance=1),
            [{'dash': 'solid', 'color': 'green', 'fillcolour': "rgba(0, 255, 0, 0.1)", 'width': 2}, # support # green = rgba(0, 255, 0, 0.1)
            {'dash': 'solid', 'color': 'red', 'fillcolour': "rgba(255, 0, 0, 0.1)", 'width': 2}], # resistance # red = rgba(255, 0, 0, 0.1)
            chart_type = 'support_resistance')

        # f.add_ta(ta.AddColumn(self.entryName), [{'color': 'yellow', 'size': 3}, {'color': 'red', 'size': 3}], chart_type='points', row=1)
        # f.add_ta(ta.AddColumn(self.stopName), [{'color': 'magenta', 'size': 5}], chart_type='points', row=1, nameCol=stopNameCol)
        # f.add_ta(ta.AddColumn(self.targetName), {'dash': 'solid', 'color': 'cyan', 'width': 3}, chart_type='lines+markers', row=1)
        # f.add_ta(ta.AddColumn(self.riskName), {'dash': 'solid', 'color': 'red', 'width': 3}, chart_type='lines+markers', row=3)
        
    def veiw_orders(self):
        pd.set_option('display.float_format', '{:.2f}'.format)
        print("===============================================")
        print (f"    Trader: {self.name} -- {self.symbol} -- {self.ls}")
        print("===============================================\n")
        print (" ---- Trade Summeries ---- ")
        display(pd.DataFrame(self.tradeSummeries))
        print (" ---- Orders to Send to IB ---- ")
        display(self.orderX.get_orders_as_df())
    
    
    def place_orders(self, delay_between_orders=1):
        self.orderX.place_orders( delay_between_orders)

f = sx.frames['1 day']
# ps.require_ta_for_all(f, pointsSpan=10)
# ps.ma_ta(f, [8, 21, 50, 200])
# ps.req_for_price_x(f, px.entryName, px.stopName, px.targetName, px.riskName, stopNameCol='StopName')
tx = TraderX('T1', ib, 'TSLA', 'LONG')
# tx.setup_columns(f.data)
  

# OR use the priceX objects to set the prices
tx.add_entry(entryx=EntryX(orderType='STP', longPriceCol='high', shortPriceCol='low', barsAgo=1, offsetPct=0.00, limitOffsetPct=0.001))
# tx.add_entry(entryx=EntryX(orderType='MKT'))

trailInitType = 'rrr'
tx.add_stop_and_target(qtyPct=25,
                         targetx=TargetX(longPriceCol='Res_1', shortPriceCol='Sup_1', offsetVal=0.50, barsAgo=1, rrIfNoTarget=2),
                         initStop=StopX(longPriceCol='LoIst_lo_3', shortPriceCol='HiIst_hi_3', offsetVal=0.50, barsAgo=1),
                         trailingStopPrices= [
                                TrailX(initType=trailInitType, initTrigVal=1, barsAgo=2, longPriceCol='FFILL_LP_lo_3', shortPriceCol='FFILL_HP_hi_3', offsetVal=0.01),
                                TrailX(initType=trailInitType, initTrigVal=2, barsAgo=2, longPriceCol='MA_cl_50',      shortPriceCol='MA_cl_50',      offsetVal=0.01),
                                TrailX(initType=trailInitType, initTrigVal=3, barsAgo=2, longPriceCol='MA_cl_21',      shortPriceCol='MA_cl_21',      offsetVal=0.01)
                    ])

tx.add_stop(qtyPct=75, 
                    initStop=StopX(name='Stop1', longPriceCol='LoIst_lo_3', shortPriceCol='HiIst_hi_3', offsetVal=0.50, barsAgo=1),
                    trailingStopPrices= [
                        TrailX(name='Stop1', initType=trailInitType, initTrigVal=1, barsAgo=2, longPriceCol='FFILL_LP_lo_3', shortPriceCol='FFILL_HP_hi_3', offsetVal=0.01),
                        TrailX(name='Stop2', initType=trailInitType, initTrigVal=2, barsAgo=2, longPriceCol='MA_cl_50',      shortPriceCol='MA_cl_50',      offsetVal=0.01),
                        TrailX(name='Stop3', initType=trailInitType, initTrigVal=3, barsAgo=2, longPriceCol='MA_cl_21',      shortPriceCol='MA_cl_21',      offsetVal=0.01)
                    ])

tx.set_columns(f.data)
tx.add_ta(f)

tx.set_orders(f.data, riskAmount=1000, outsideRth=True)
tx.place_orders()

# tx.orderX.orders

# for g  in tx.pricexGroups:
#     print(g[0], g[1].trails[0])
#     print(g[0], g[1].stop)
#     print(g[0], g[1].target)
#     print(g[0], g[1].status)

f.data.columns
tx.veiw_orders()

Limit Entry Price: 429.54, Entry Price: 429.11
    Trader: T1 -- TSLA -- LONG

 ---- Trade Summeries ---- 


,status,outsideRth,isOutSideRth,qtyPct,qty,riskAmount,totalValue,potentialLoss,entryPrice,entryLimitPrice,stopPrice,targetPrice,entryOrderType,actualEntryOrderType
0,TARGET_PRICE_FOUND,True,True,25,6,250.00,2574.66,216.66,429.11,429.54,393.00,488.54,STP LMT,
1,INIT_STOP_PRICE_FOUND,True,True,75,20,750.00,8582.20,722.20,429.11,429.54,393.00,NaN,STP LMT,


 ---- Orders to Send to IB ---- 


,parentId,orderId,clientId,permId,action,totalQuantity,orderType,ocaGroup,ocaType,orderRef,lmtPrice,auxPrice,outsideRth,tif,goodTillDate,transmit
0,0,843,12,816064458,BUY,6.00,STP LMT,,0,Strat1_Entry_1,429.54,429.11,True,,,False
1,843,844,12,816064459,SELL,6.00,STP,,0,Strat1_Stop_1_pct25,0.00,393.00,False,,,False
2,843,845,12,816064460,SELL,6.00,LMT,,0,Strat1_Tget_1_pct25,488.54,0.00,False,,,True
3,0,846,12,816064461,BUY,19.00,STP LMT,,0,Strat1_Entry_1,429.54,429.11,True,,,False
4,846,847,12,816064462,SELL,19.00,STP,,0,Strat1_Stop_2_pct75,0.00,393.00,False,,,True


In [5]:
opntrades = tx.ib.openTrades()
# opntrades = tx.ib.trades()
opntrades

[Trade(contract=Stock(symbol='TSLA', exchange='SMART', currency='USD'), order=StopOrder(orderId=812, clientId=12, permId=816064441, action='BUY', totalQuantity=6.0, lmtPrice=429.54, auxPrice=429.11, orderRef='Strat1_Entry_1', transmit=False, outsideRth=True), orderStatus=OrderStatus(orderId=812, status='PreSubmitted', filled=0.0, remaining=6.0, avgFillPrice=0.0, permId=816064441, parentId=0, lastFillPrice=0.0, clientId=12, whyHeld='trigger', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2025, 1, 18, 19, 3, 11, 950165, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2025, 1, 18, 19, 3, 12, 272795, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='')]),
 Trade(contract=Stock(symbol='TSLA', exchange='SMART', currency='USD'), order=StopOrder(orderId=813, clientId=12, permId=816064442, action='SELL', totalQuantity=6.0, lmtPrice=0.0, auxPrice=393.0, orderRef='Strat1_Stop_1_pct25', transmit=False, pa

In [18]:
tx.orderX.get_orders_status_as_df()

,orderId,status,filled,remaining,avgFillPrice,permId,parentId,lastFillPrice,clientId,whyHeld,mktCapPrice
0,821,PreSubmitted,0.00,6.00,0.00,816064447,0,0.00,12,trigger,0.00
1,822,PreSubmitted,0.00,6.00,0.00,816064448,821,0.00,12,"child,trigger",0.00
2,823,PreSubmitted,0.00,6.00,0.00,816064449,821,0.00,12,child,0.00
3,824,PreSubmitted,0.00,19.00,0.00,816064450,0,0.00,12,trigger,0.00
4,825,PreSubmitted,0.00,19.00,0.00,816064451,824,0.00,12,"child,trigger",0.00


In [4]:
def get_remaining_by_orderid(df, order_id):
    s = df.loc[df['orderId'] == order_id, 'remaining'] # .iloc[0]
    if len(s) == 0:
        return None
    return s.iloc[0]

get_remaining_by_orderid(tx.orderX.get_orders_status_as_df(), 23)

In [9]:

def update_filled_status(tx):

    def get_remaining_by_orderid(df, order_id):
        s = df.loc[df['orderId'] == order_id, 'remaining']
        if len(s) == 0:
            return None
        return s.iloc[0]

    order_status_df = tx.orderX.get_orders_status_as_df()

    for pct, pricex in tx.pricexGroups:
        entry_remainng = get_remaining_by_orderid(order_status_df, pricex.entry.ibId)
        stop_remainng = get_remaining_by_orderid(order_status_df, pricex.stop.ibId)
        tget_remainng = get_remaining_by_orderid(order_status_df, pricex.target.ibId)
        print(f"Entry: {entry_remainng}, Stop: {stop_remainng}, Target: {tget_remainng}")

        pricex.entry.has_triggered(forceTrigger=entry_remainng == 0)
        pricex.stop.has_triggered(forceTrigger=stop_remainng == 0)
        pricex.target.has_triggered(forceTrigger=tget_remainng == 0)
        print(f"Entry orderFilled: {pricex.entry.orderFilled}, Stop orderFilled: {pricex.stop.orderFilled}, Target orderFilled: {pricex.target.orderFilled}")



    
update_filled_status(tx)

Entry: 6.0, Stop: 6.0, Target: 6.0
Entry orderFilled: False, Stop orderFilled: False, Target orderFilled: False
Entry: 19.0, Stop: 19.0, Target: None
Entry orderFilled: False, Stop orderFilled: False, Target orderFilled: False


In [5]:
tx.orderX.orders
is_rth = my_ib_utils.is_within_trading_hours(ib, 'TSLA', 'SMART')
print(f"Is RTH: {is_rth}")
print(f"Is RTH: {not is_rth}")


Is RTH: False
Is RTH: True


In [4]:
for g  in tx.pricexGroups:
    print(g[0], g[1].trails[0])
    print(g[0], g[1].stop)
    print(g[0], g[1].target)
    print(g[0], g[1].status)

    print(g[0], id(g[1].stop))
# tx.veiw_orders()
# tx.orderX.get_orders_status_as_df()

25 TrailX(name='T1_Strat1_Tra1', ls='LONG', priceCol='FFILL_LP_lo_3', price=nan, offsetVal=0.01, offsetPct=0.0, barsAgo=2, initType='rrr', initTrigVal=1, longPriceCol='FFILL_LP_lo_3', shortPriceCol='FFILL_HP_hi_3')
25 StopX(name='T1_Strat1_Sto2', ls='LONG', priceCol='LoIst_lo_3', price=377.29, offsetVal=0.5, offsetPct=0.0, barsAgo=1, longPriceCol='LoIst_lo_3', shortPriceCol='HiIst_hi_3')
25 TargetX(name='T1_Strat1_Tar2', ls='LONG', priceCol='Res_1', price=488.54, offsetVal=0.5, offsetPct=0.0, barsAgo=1, longPriceCol='Res_1', shortPriceCol='Sup_1', rrIfNoTarget=2)
25 TARGET_PRICE_FOUND
25 2427199229184
75 TrailX(name='T1_Strat2_Tra0', ls='LONG', priceCol='FFILL_LP_lo_3', price=nan, offsetVal=0.01, offsetPct=0.0, barsAgo=2, initType='rrr', initTrigVal=1, longPriceCol='FFILL_LP_lo_3', shortPriceCol='FFILL_HP_hi_3')
75 StopX(name='T1_Strat2_Sto1', ls='LONG', priceCol='LoIst_lo_3', price=377.29, offsetVal=0.5, offsetPct=0.0, barsAgo=1, longPriceCol='LoIst_lo_3', shortPriceCol='HiIst_hi_3')


In [12]:
tx.pricexGroups[0][1].object_count
tx.pricexGroups[0][1].stop
tx.orderX.orders
tx.pricexGroups[0][1]

PriceX(name='T1_Strat1', ls='LONG', includeTarget=True, entry=EntryX(name='T1_Strat2_Ent1', ls='LONG', priceCol='close', price=428.98, offsetVal=0.0, offsetPct=0.0, barsAgo=0, orderType='MKT', longPriceCol='close', shortPriceCol='close'), stop=StopX(name='T1_Strat1_Sto2', ls='LONG', priceCol='LoIst_lo_3', price=393.0, offsetVal=0.5, offsetPct=0.0, barsAgo=1, longPriceCol='LoIst_lo_3', shortPriceCol='HiIst_hi_3'), target=TargetX(name='T1_Strat1_Tar2', ls='LONG', priceCol='Res_1', price=488.54, offsetVal=0.5, offsetPct=0.0, barsAgo=1, longPriceCol='Res_1', shortPriceCol='Sup_1', rrIfNoTarget=2), trails=[TrailX(name='T1_Strat1_Tra1', ls='LONG', priceCol='FFILL_LP_lo_3', price=nan, offsetVal=0.01, offsetPct=0.0, barsAgo=2, initType='rrr', initTrigVal=1, longPriceCol='FFILL_LP_lo_3', shortPriceCol='FFILL_HP_hi_3'), TrailX(name='T1_Strat1_Tra2', ls='LONG', priceCol='MA_cl_50', price=nan, offsetVal=0.01, offsetPct=0.0, barsAgo=2, initType='rrr', initTrigVal=2, longPriceCol='MA_cl_50', shortPr

In [3]:
f.plot()

In [ ]:
# changed delay to shift in new_stop method. this means the number of bars to shift to get the value. 0 means current bar, 1 means previous bar, etc
# Risk should be calcualted upon init of the stop manager and stored as a set value.  The rrRatio is computed on the fly as the price changes and uses the original risk value compared to the current reward value (current price - entry price)
# we need to add each new_stop type as a separet method that is called when the trigger is met.  This method should set the new stop value and offset values. the reason the ways of getting trig values is too varied to have a single method.
# eg new_stop_rRatio, new_stop_accel, new_stop_priceDist 
# remove all the doc strings for now whilst debugging and testing the code


In [18]:
import pandas as pd
import numpy as np
from dataclasses import dataclass

def generate_signal(df, n):
    lower_high = df['high'] < df['high'].shift(n)
    breaks_prev_bar = df['close'] > df['low'].shift(1)

    df['signal'] = np.nan
    df['signal'] = np.where(lower_high & breaks_prev_bar, True, np.nan)

    return df


@dataclass
class RiskRewardRatio:
    entry: float = 0
    stop: float = 0
    price: float = 0
    risk: float = 0
    reward: float = 0
    rrRatio: float = 0

    def get_ratio(self):
        self.risk = self.entry - self.stop
        self.reward = self.price - self.entry
        self.rrRatio = round(self.reward / self.risk, 1)
        print(f"Entry: {self.entry}, Stop: {self.stop}, Price: {self.price}, Risk: {self.risk}, Reward: {self.reward}, RR Ratio: {self.rrRatio}")
        return self.rrRatio
    
def generate_risk_at_points(df):
    df['risk'] = np.where(df['LP_lo_10'] > 0, df['high'] - df['low'], np.nan)   
    df['rRatio'] = np.nan





    for i, row in df.iterrows():
        dfx = df.loc[:i]
        if len(dfx) < 2:
            continue
        if dfx['signal'].iat[-1] > 0: 
            rrr = RiskRewardRatio()
            rrr.entry = dfx['close'].iat[-2]
            rrr.stop = dfx['LP_lo_10'].iat[-2]
            rrr.price = dfx['close'].iat[-1]
            dfx.at[i, 'rRatio'] = rrr.get_ratio()
    return dfx

f.data = generate_signal(f.data, 3)
f.data = generate_risk_at_points(f.data)
# df.loc[df['signal'] == True][['risk']].dropna()
f.data[['signal', 'risk', 'rRatio', 'StopX']].dropna(how='all').tail(50)

Entry: 16.53, Stop: nan, Price: 16.39, Risk: nan, Reward: -0.14000000000000057, RR Ratio: nan
Entry: 16.39, Stop: 16.05, Price: 16.4, Risk: 0.33999999999999986, Reward: 0.00999999999999801, RR Ratio: 0.0
Entry: 16.46, Stop: nan, Price: 16.35, Risk: nan, Reward: -0.10999999999999943, RR Ratio: nan
Entry: 16.35, Stop: nan, Price: 17.09, Risk: nan, Reward: 0.7399999999999984, RR Ratio: nan
Entry: 16.09, Stop: nan, Price: 16.33, Risk: nan, Reward: 0.23999999999999844, RR Ratio: nan
Entry: 16.33, Stop: 16.03, Price: 17.02, Risk: 0.29999999999999716, Reward: 0.6900000000000013, RR Ratio: 2.3
Entry: 25.05, Stop: nan, Price: 24.01, Risk: nan, Reward: -1.0399999999999991, RR Ratio: nan
Entry: 25.19, Stop: nan, Price: 25.41, Risk: nan, Reward: 0.21999999999999886, RR Ratio: nan
Entry: 23.4, Stop: nan, Price: 22.74, Risk: nan, Reward: -0.6600000000000001, RR Ratio: nan
Entry: 22.74, Stop: nan, Price: 23.59, Risk: nan, Reward: 0.8500000000000014, RR Ratio: nan
Entry: 24.04, Stop: nan, Price: 23.81

,signal,risk,rRatio,StopX
date,,,,
2024-09-27,1.0,NaN,NaN,NaN
2024-10-16,1.0,NaN,NaN,NaN
2024-10-17,1.0,NaN,NaN,NaN
2024-10-18,1.0,NaN,NaN,NaN
2024-10-31,NaN,2.42,NaN,40.9
2024-11-01,1.0,NaN,0.5,40.9
2024-11-04,NaN,NaN,NaN,40.9
2024-11-05,NaN,NaN,NaN,40.9
2024-11-06,NaN,NaN,NaN,40.9


In [1]:

# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

from ib_insync import *
util.startLoop()
import stock

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=12)



<IB connected to 127.0.0.1:7496 clientId=12>

In [2]:
from my_ib_utils import *
inhours = is_within_trading_hours(ib, 'TSLA', 'SMART')
inhours

False

In [4]:
ousideRth = True
inhours = True


# senarios for placing orders
# 1. allowedOutsideRth = True and inhours = True
# 2. allowedOutsideRth = True and inhours = False
# 3. allowedOutsideRth = False and inhours = True
# 4. allowedOutsideRth = False and inhours = False

# entry order types = MKT, STP

# if outside RTH then set limit price 
# if MKT then no entry price required 
# if STP then entry price is required


if ousideRth and inhours:
    print("Outside RTH and within trading hours")
    print("ORDER TYPEs = MKT, LMT, STP, STP LMT")
elif ousideRth and not inhours:
    print("Outside RTH and outside trading hours")
    print("ORDER TYPEs = LMT, STP LMT")

Outside RTH and within trading hours


In [8]:
from datetime import datetime, timezone, time
import pytz

def is_within_trading_hours(ib, symbol: str, exchange: str = 'SMART', debug: bool = False) -> bool:
    """
    Check if a given stock is actually trading right now using IB's trading schedule.
    
    Args:
        ib: The IB instance from ib_insync
        symbol: The stock symbol (e.g., 'TSLA')
        exchange: The exchange to use (default: 'SMART')
        debug: If True, prints detailed debugging information (default: False)
    """
    # Exchange timezone mapping
    EXCHANGE_TIMEZONES = {
        'NYSE': 'America/New_York',
        'NASDAQ': 'America/New_York',
        'ISLAND': 'America/New_York',
        'ARCA': 'America/New_York',
        'IEX': 'America/New_York',
        'BATS': 'America/New_York',
        'LSE': 'Europe/London',
        'TSE': 'Asia/Tokyo',
    }
    
    try:
        # Create and qualify contract
        contract = Stock(symbol, exchange, 'USD')
        qualified_contract = ib.qualifyContracts(contract)[0]
        
        # Get exchange timezone
        exchange_name = qualified_contract.primaryExchange or qualified_contract.exchange
        timezone_str = EXCHANGE_TIMEZONES.get(exchange_name, 'America/New_York')
        exchange_tz = pytz.timezone(timezone_str)
        
        if debug:
            print("\n=== Trading Hours Debug Info ===")
            print(f"Symbol: {symbol}")
            print(f"Exchange: {exchange}")
            print(f"Qualified Exchange: {qualified_contract.exchange}")
            print(f"Primary Exchange: {qualified_contract.primaryExchange}")
            print(f"Exchange Timezone: {timezone_str}")
            
        # Get trading schedule
        contract_details = ib.reqContractDetails(qualified_contract)[0]
        schedule = contract_details.liquidHours
        
        if not schedule:
            schedule = contract_details.tradingHours
            if debug:
                print("\nUsing tradingHours (liquidHours not available)")
        else:
            if debug:
                print("\nUsing liquidHours")
                
        if debug:
            print(f"Raw Schedule: {schedule}")
            
        if not schedule:
            raise ValueError(f"Could not get trading schedule for {symbol}")
            
        # Current time in UTC and exchange timezone
        now_utc = datetime.now(timezone.utc)
        now_exchange = now_utc.astimezone(exchange_tz)
        
        if debug:
            print(f"\nCurrent UTC time: {now_utc.strftime('%Y-%m-%d %H:%M:%S %Z')}")
            print(f"Current {exchange_name} time: {now_exchange.strftime('%Y-%m-%d %H:%M:%S %Z')}")
            print("\nParsed Trading Windows:")
        
        # Parse the schedule
        for segment in schedule.split(';'):
            if '-' in segment and len(segment) == 9:  # Time range format: hhmm-hhmm
                start_str, end_str = segment.split('-')
                
                # Convert time strings to datetime objects (in UTC)
                start_time = datetime.now(timezone.utc).replace(
                    hour=int(start_str[:2]),
                    minute=int(start_str[2:]),
                    second=0,
                    microsecond=0
                )
                
                end_time = datetime.now(timezone.utc).replace(
                    hour=int(end_str[:2]),
                    minute=int(end_str[2:]),
                    second=0,
                    microsecond=0
                )
                
                # Convert to exchange timezone for debug output
                start_time_exchange = start_time.astimezone(exchange_tz)
                end_time_exchange = end_time.astimezone(exchange_tz)
                
                if debug:
                    print(f"\nTrading Window:")
                    print(f"UTC: {start_time.strftime('%H:%M')} - {end_time.strftime('%H:%M')}")
                    print(f"{exchange_name}: {start_time_exchange.strftime('%H:%M')} - {end_time_exchange.strftime('%H:%M')}")
                    print(f"Current time within this window? {start_time <= now_utc <= end_time}")
                
                if start_time <= now_utc <= end_time:
                    if debug:
                        print("\n=== RESULT: MARKET IS OPEN ===")
                    return True
        
        if debug:
            print("\n=== RESULT: MARKET IS CLOSED ===")
        return False
        
    except Exception as e:
        error_msg = f"Error checking trading hours for {symbol}: {str(e)}"
        if debug:
            print(f"\n=== ERROR ===\n{error_msg}")
        raise ValueError(error_msg)
    
is_within_trading_hours(ib, 'AAPL', debug=True)


=== Trading Hours Debug Info ===
Symbol: AAPL
Exchange: SMART
Qualified Exchange: SMART
Primary Exchange: ISLAND
Exchange Timezone: America/New_York

Using liquidHours
Raw Schedule: 20250118:CLOSED;20250119:CLOSED;20250120:CLOSED;20250121:0930-20250121:1600;20250122:0930-20250122:1600;20250123:0930-20250123:1600

Current UTC time: 2025-01-18 17:12:52 UTC
Current ISLAND time: 2025-01-18 12:12:52 EST

Parsed Trading Windows:

=== RESULT: MARKET IS CLOSED ===


False